In [1]:
from pathlib import Path
from ace_rl.core.cube_builder import CubeBuilderConfig, build_raw_cube

root = Path("../../../work/processed/binance").expanduser()
cfg = CubeBuilderConfig(
    per_symbol_root=root / "data" / "zarr_per_symbol",
    # chưa có meta -> None; builder sẽ suy luận
    calendar_path=None,
    feature_order_path=None,
    symbols=["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT"],
    use_dask=False,
    # Quan trọng: cắt block theo giao thời gian
    calendar_mode="range_intersection",  # mặc định đã là mode này
    output_path=root / "data" / "zarr_cube_intersection",  # tuỳ chọn
)

cube = build_raw_cube(cfg)
print(cube)


/home/kylh/phd/tw_fin_rl/ace_rl/core/cube_builder.py:114: UserWarning: no explicit representation of timezones available for np.datetime64
  raw_times = np.asarray(group["time"][:], dtype="datetime64[ns]")


<xarray.Dataset> Size: 501MB
Dimensions:  (symbol: 4, time: 2662909, feature: 9)
Coordinates:
  * symbol   (symbol) object 32B 'BTCUSDT' 'ETHUSDT' 'BNBUSDT' 'SOLUSDT'
  * time     (time) datetime64[ns] 21MB 2020-08-11T06:00:00 ... 2025-09-04T11...
  * feature  (feature) <U13 468B 'Open' 'High' ... 'TakerBuyQuote'
Data variables:
    values   (time, symbol, feature) float32 383MB dask.array<chunksize=(16384, 4, 9), meta=np.ndarray>
    mask     (time, symbol, feature) bool 96MB dask.array<chunksize=(16384, 4, 9), meta=np.ndarray>
Attributes:
    calendar_source:       inferred:range_intersection
    feature_order_source:  inferred


In [2]:
cube


<xarray.Dataset> Size: 501MB
Dimensions:  (symbol: 4, time: 2662909, feature: 9)
Coordinates:
  * symbol   (symbol) object 32B 'BTCUSDT' 'ETHUSDT' 'BNBUSDT' 'SOLUSDT'
  * time     (time) datetime64[ns] 21MB 2020-08-11T06:00:00 ... 2025-09-04T11...
  * feature  (feature) <U13 468B 'Open' 'High' ... 'TakerBuyQuote'
Data variables:
    values   (time, symbol, feature) float32 383MB dask.array<chunksize=(16384, 4, 9), meta=np.ndarray>
    mask     (time, symbol, feature) bool 96MB dask.array<chunksize=(16384, 4, 9), meta=np.ndarray>
Attributes:
    calendar_source:       inferred:range_intersection
    feature_order_source:  inferred

In [3]:
import xarray as xr
import numpy as np

cube_value = cube["values"]  # (time, symbol, feature)
close = cube_value.sel(feature="Close")

In [4]:

# 1. Cắt fragment độ dài 240 phút
fragment = close.rolling(time=240).construct("fragment_step").dropna("time")
fragment_id = fragment.get_index("time")  # hoặc tự build id

print(fragment_id)

DatetimeIndex(['2020-08-11 09:59:00', '2020-08-11 10:00:00',
               '2020-08-11 10:01:00', '2020-08-11 10:02:00',
               '2020-08-11 10:03:00', '2020-08-11 10:04:00',
               '2020-08-11 10:05:00', '2020-08-11 10:06:00',
               '2020-08-11 10:07:00', '2020-08-11 10:08:00',
               ...
               '2025-09-04 11:43:00', '2025-09-04 11:44:00',
               '2025-09-04 11:45:00', '2025-09-04 11:46:00',
               '2025-09-04 11:47:00', '2025-09-04 11:48:00',
               '2025-09-04 11:49:00', '2025-09-04 11:50:00',
               '2025-09-04 11:51:00', '2025-09-04 11:52:00'],
              dtype='datetime64[ns]', name='time', length=2662430, freq=None)


In [5]:
fragment 

<xarray.DataArray 'values' (time: 2662430, symbol: 4, fragment_step: 240)> Size: 10GB
dask.array<getitem, shape=(2662430, 4, 240), dtype=float32, chunksize=(16336, 1, 240), chunktype=numpy.ndarray>
Coordinates:
  * symbol   (symbol) object 32B 'BTCUSDT' 'ETHUSDT' 'BNBUSDT' 'SOLUSDT'
  * time     (time) datetime64[ns] 21MB 2020-08-11T09:59:00 ... 2025-09-04T11...
    feature  <U13 52B 'Close'
Dimensions without coordinates: fragment_step

In [6]:
# bỏ phần tử đầu (vì diff đã mất phần tử đầu)
numer = fragment.diff("fragment_step")                          # (time, symbol, 239)
denom = fragment.isel(fragment_step=slice(None, -1))            # (time, symbol, 239)
returns = numer / denom                                         # pct change từng bước
# hoặc log return: returns = np.log(fragment.isel(fragment_step=slice(1, None)) / denom)

returns 

<xarray.DataArray 'values' (time: 2662430, symbol: 4, fragment_step: 239)> Size: 10GB
dask.array<truediv, shape=(2662430, 4, 239), dtype=float32, chunksize=(16336, 1, 239), chunktype=numpy.ndarray>
Coordinates:
  * symbol   (symbol) object 32B 'BTCUSDT' 'ETHUSDT' 'BNBUSDT' 'SOLUSDT'
  * time     (time) datetime64[ns] 21MB 2020-08-11T09:59:00 ... 2025-09-04T11...
    feature  <U13 52B 'Close'
Dimensions without coordinates: fragment_step

In [ ]:
up_thr = 0.0015      # 0.15%/step (điều chỉnh theo data)
down_thr = -0.0015
vol_thr = 0.01       # 1%/step

mean_ret = returns.mean("fragment_step")
vol = returns.std("fragment_step")

q_hi = mean_ret.quantile(0.7)
q_lo = mean_ret.quantile(0.3)
vol_thr = vol.quantile(0.75)

regime = xr.full_like(mean_ret, "sideway", dtype=object)
regime = regime.where(~((mean_ret > q_hi) & (vol <= vol_thr)), "bull")
regime = regime.where(~((mean_ret < q_lo) & (vol <= vol_thr)), "bear")
regime = regime.where(~(vol > vol_thr), "volatile")


regime 

<xarray.DataArray 'values' (time: 2662430, symbol: 4)> Size: 85MB
dask.array<where, shape=(2662430, 4), dtype=object, chunksize=(16336, 1), chunktype=numpy.ndarray>
Coordinates:
  * symbol   (symbol) object 32B 'BTCUSDT' 'ETHUSDT' 'BNBUSDT' 'SOLUSDT'
  * time     (time) datetime64[ns] 21MB 2020-08-11T09:59:00 ... 2025-09-04T11...
    feature  <U13 52B 'Close'

In [11]:
cats = ["bull", "bear", "sideway"]  # danh sách regime bạn đang dùng

counts_by_symbol = xr.Dataset(
    {cat: (regime == cat).sum(dim="time") for cat in cats}
).to_array("regime")  # DataArray: dims ("regime", "symbol")

counts_by_symbol  # cần .compute() nếu đang dùng dask lazy


<xarray.DataArray (regime: 3, symbol: 4)> Size: 96B
dask.array<stack, shape=(3, 4), dtype=int64, chunksize=(1, 1), chunktype=numpy.ndarray>
Coordinates:
  * symbol   (symbol) object 32B 'BTCUSDT' 'ETHUSDT' 'BNBUSDT' 'SOLUSDT'
    feature  <U13 52B 'Close'
  * regime   (regime) object 24B 'bull' 'bear' 'sideway'

In [13]:
counts = counts_by_symbol.compute()  # DataArray dims: ("regime", "symbol")

for symbol in counts.symbol.values:
    print(symbol)
    row = counts.sel(symbol=symbol)
    for cat, val in zip(row.regime.values, row.values):
        print(f"  {cat}: {int(val)}")


BTCUSDT
  bull: 0
  bear: 0
  sideway: 2662430
ETHUSDT
  bull: 0
  bear: 2
  sideway: 2662428
BNBUSDT
  bull: 0
  bear: 4
  sideway: 2662426
SOLUSDT
  bull: 20
  bear: 5
  sideway: 2662405


In [16]:
import itertools
import numpy as np
import pandas as pd
import xarray as xr

# --- 1. Fragments & returns --------------------------------------------------

def build_fragments(close_da: xr.DataArray, window: int) -> xr.DataArray:
    frag = (
        close_da
        .rolling(time=window)
        .construct("fragment_step")
        .dropna("time")
        .assign_coords(fragment_step=np.arange(window))
    )
    return frag

def fragment_returns(fragment: xr.DataArray, mode: str = "pct") -> xr.DataArray:
    # cắt fragment thành hai lát cùng kích thước 239 trên chiều fragment_step
    p_tm1 = fragment.isel(fragment_step=slice(None, -1))
    p_t   = fragment.isel(fragment_step=slice(1, None))

    if mode == "log":
        returns = xr.apply_ufunc(np.log, p_t / p_tm1)
    else:
        returns = (p_t - p_tm1) / p_tm1

    # gán lại chỉ số fragment_step (0..window-2) để về đúng kích thước
    returns = returns.assign_coords(
        fragment_step=np.arange(returns.sizes["fragment_step"], dtype=np.int32)
    )
    return returns


# --- 2. Threshold helpers ----------------------------------------------------

def make_symbol_threshold(data: xr.DataArray, mode: str, value: float) -> xr.DataArray:
    if mode == "quantile":
        thr = data.groupby("symbol").quantile(value)
    else:  # "abs"
        thr = xr.DataArray(
            np.full(data.sizes["symbol"], value, dtype=np.float32),
            coords={"symbol": data.symbol},
            dims=["symbol"],
        )
    return thr.astype(np.float32)

REGIME_CODES = {"sideway": 0, "bull": 1, "bear": 2, "volatile": 3}

def classify_regime(mean_ret: xr.DataArray,
                    vol: xr.DataArray,
                    params: dict) -> xr.DataArray:
    hi = make_symbol_threshold(
        mean_ret, params["return_hi_mode"], params["return_hi_value"]
    )
    lo = make_symbol_threshold(
        mean_ret, params["return_lo_mode"], params["return_lo_value"]
    )
    vol_thr = make_symbol_threshold(
        vol, params["vol_mode"], params["vol_value"]
    )

    regime = xr.full_like(mean_ret, REGIME_CODES["sideway"], dtype=np.int8)

    # assign bull/bear first
    bull_mask = (mean_ret > hi) & (vol <= vol_thr)
    bear_mask = (mean_ret < lo) & (vol <= vol_thr)
    regime = xr.where(bull_mask, REGIME_CODES["bull"], regime)
    regime = xr.where(bear_mask, REGIME_CODES["bear"], regime)

    # volatile overrides other states
    if params.get("flag_volatile", True):
        vol_mask = vol > vol_thr
        regime = xr.where(vol_mask, REGIME_CODES["volatile"], regime)

    return regime


In [ ]:
WINDOW = 240  # 4 hours of 1‑minute bars
fragments = build_fragments(close, WINDOW)
returns = fragment_returns(fragments, mode="pct").persist()  # cache once

mean_ret = returns.mean("fragment_step").persist()
vol = returns.std("fragment_step").persist()


In [ ]:




def classify(mean_ret, vol, up=0.002, down=-0.002, vol_thr=0.005):
    regime = xr.full_like(mean_ret, "sideway", dtype=object)
    regime = regime.where(~((mean_ret > up) & (vol < vol_thr)), "bull")
    regime = regime.where(~((mean_ret < down) & (vol < vol_thr)), "bear")
    return regime

regime = classify(mean_ret, vol)

# 3. Lưu metadata
meta = xr.Dataset(
    {
        "mean_ret": mean_ret,
        "vol": vol,
        "regime": regime,
    },
    coords={"time": mean_ret.time, "symbol": mean_ret.symbol},
)


AlignmentError: cannot reindex or align along dimension 'fragment_step' because of conflicting dimension sizes: {240, 239}

In [ ]:
fragment_id

DatetimeIndex(['2020-08-12 05:59:00', '2020-08-12 06:00:00',
               '2020-08-12 06:01:00', '2020-08-12 06:02:00',
               '2020-08-12 06:03:00', '2020-08-12 06:04:00',
               '2020-08-12 06:05:00', '2020-08-12 06:06:00',
               '2020-08-12 06:07:00', '2020-08-12 06:08:00',
               ...
               '2025-09-04 11:43:00', '2025-09-04 11:44:00',
               '2025-09-04 11:45:00', '2025-09-04 11:46:00',
               '2025-09-04 11:47:00', '2025-09-04 11:48:00',
               '2025-09-04 11:49:00', '2025-09-04 11:50:00',
               '2025-09-04 11:51:00', '2025-09-04 11:52:00'],
              dtype='datetime64[ns]', name='time', length=2660030, freq=None)